In [15]:
import os

Reading API Keys

In [16]:
#Reading OpenAI Key
OPENAI_API_KEY=input("Enter OpenAI API Key")
#Reading Langchain API Key
LANGCHAIN_API_KEY=input("Enter Langchain API Key")

In [51]:
#Enabiling Langchain Tracing
LANGCHAIN_TRACING_V2= "true"
#OpenAI Model
OPENAI_MODEL="gpt-4o-mini"
LANGCHAIN_TRACING_V2="true"


In [ ]:
# installing langchain_openai package
! pip install langchain_openai

In [52]:
# initializing llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model=OPENAI_MODEL,openai_api_key=OPENAI_API_KEY,temperature=0.5)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1272998e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12729a3c0>, root_client=<openai.OpenAI object at 0x12728eed0>, root_async_client=<openai.AsyncOpenAI object at 0x1272997f0>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
# Installing Python package for SQLDatabase
! pip install langchain-community==0.2.9
! pip install sqlalchemy==2.0.28

In [53]:
#initializing Database object
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///baggage_tracking.db")
db

In [54]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.outputs import  Generation
class SQlExtractParser(BaseGenerationOutputParser[str]):
    """
    A parser for SQL output.
    """

    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """
        Parse the SQL query input and return the extracted SQL query.

        Args:
            sql_query_input (str): The input SQL query string.
            partial (bool, optional): Whether the input is a partial query. Defaults to False.

        Returns:
            str: The extracted SQL query.
        """
        if not result:
            raise OutputParserException("Input SQL query is empty")
        
        print(f"Incoming query {result[0]}")

        if 'SQLQuery:' in result[0].text:
            sql_query = result[0].text.split('SQLQuery:', 1)[1].strip()
            print(f"Extracted SQL query: {sql_query}")
        else:
            sql_query = result[0].text

        return sql_query

In [55]:
sql_examples = [
    {
        "input": "Show bag tag numbers for the PNR number AAA100",
        "query": "select bag_tag_number from baggage_details where pnr = 'AAA100';"
    },
    {
        "input": "Find the baggage details of the flight originating from JFK and destination is SFO",
        "query": "select * from baggage_details where origin ='JFK' and destination ='SFO';"
    },
    {
        "input": "Find scanning details of bag tag number 001234567890",
        "query": "SELECT * from baggage_scanning where bag_tag_number = '001234567890';"
    },
    {
        "input": "Find the passengers in flight BC101",
        "query": "SELECT * from passenger_details where passenger_flight_number = 'BC101';"
    },
    {
        "input": "Find the passenger details for the bag tag number 001234567890",
        "query": "SELECT * from passenger_details where bag_tag_number = '001234567890'"
    },
    {
        "input": "Find rush bags in flight AB100",
        "query": "SELECT * from baggage_details where rush_bag='Yes' and flight_number = 'AB100'"
    }
]

In [56]:
from langchain_core.prompts import ChatPromptTemplate
sql_example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai","{query}")
    ]
)

In [37]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate
sql_few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=sql_example_prompt,
    examples=sql_examples,
    input_variables=["input"]
)

In [57]:
SQL_QUERY_PROMPT=ChatPromptTemplate.from_messages(
    [
        ("system",""" You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
        Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
        Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
        Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
        Pay attention to use date('now') function to get the current date, if the question involves "today".

        Use the following format:

        Question: Question here
        SQLQuery: SQL Query to run
        SQLResult: Result of the SQLQuery
         
        {top_k}

        Only use the following tables:
        {table_info}"""),
        sql_few_shot_prompt,
        ("human","{input}")
    ]
)

In [58]:
from langchain_core.prompts import PromptTemplate
ANSWER_PROMPTS = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.
     Combine the the Question and SQL Result to generate a meaningful answer to the user.
     The answer should be in the format of a sentence.
     The answer should be a direct response to the user question.
     Respond to the question as an experienced airline ground staff

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

In [59]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
# Create a query chain
query_chain = create_sql_query_chain(llm, db,prompt=SQL_QUERY_PROMPT) | SQlExtractParser()

# Create an execution chain
execute_query = QuerySQLDataBaseTool(db=db)
execute_chain = itemgetter("query") | execute_query

# Create an answer chain
answer_chain = ANSWER_PROMPTS | llm | StrOutputParser()

# Combine the chains
final_chain = (
    RunnablePassthrough()
    .assign(query=query_chain)
    .assign(result=execute_chain)
    | answer_chain
)

# Invoke the chain with the user's query
input_data = {
    "question": "On an average how many passengers are there in a flight?",
    "dialect": "sqlite",
    "top_k":2,
    "table_info": db.get_table_info(),
}
#answer=query_chain.invoke(input_data)
answer = final_chain.invoke(input_data)


Incoming query text='SELECT AVG(passenger_count) as average_passengers FROM (\n    SELECT COUNT(*) as passenger_count \n    FROM passenger_details \n    GROUP BY passenger_flight_number\n);'


In [61]:
answer

'On average, there are approximately 1.25 passengers per flight.'